In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn import metrics
from lightgbm import plot_importance
from matplotlib import pyplot
import matplotlib.pyplot as plt

In [ ]:
#train = pd.read_csv('ref/train_featureV1.csv')
#test = pd.read_csv('ref/test_featureV1.csv')

In [ ]:
train = pd.read_csv('train_feature.csv')
test = pd.read_csv('test_feature.csv')

In [ ]:
dtrain = lgb.Dataset(train.drop(['uid','label'],axis=1),label=train.label)
dtest = lgb.Dataset(test.drop(['uid'],axis=1))

In [ ]:
lgb_params =  {
    'boosting_type': 'gbdt',
    'is_unbalance' : True,
    'objective': 'binary',
    'is_training_metric': False,
    'min_data_in_leaf': 100,
    'num_leaves': 63,
    'learning_rate': 0.03,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq' : 1,
    'colsample_bytree' : 0.8,
    'verbosity' : 0,
    'n-estimators' : 500,
}    

In [ ]:
def evalMetric(preds,dtrain):
    
    label = dtrain.get_label()
    
    pre = pd.DataFrame({'preds':preds,'label':label})
    pre = pre.sort_values(by='preds',ascending=False)
    
    auc = metrics.roc_auc_score(pre.label,pre.preds)

    pre.preds=pre.preds.map(lambda x: 1 if x>=0.5 else 0)

    f1 = metrics.f1_score(pre.label,pre.preds)
    
    res = 0.6*auc+0.4*f1
    
    return 'res',res,True

In [ ]:
lgb.cv(lgb_params,dtrain,feval=evalMetric,early_stopping_rounds=50,verbose_eval=10,num_boost_round=1000,nfold=5,metrics=['evalMetric'])

In [ ]:
model = lgb.train(lgb_params,dtrain,feval=evalMetric,early_stopping_rounds=100,verbose_eval=10,num_boost_round=10000,valid_sets=dtrain)

In [ ]:
#fig, ax = plt.subplots(figsize=(15, 15))
#plot_importance(model,ax=ax)
#idx = np.where(model.feature_importance()<5)
#print(model.feature_importance())
#print(idx)

In [ ]:
#new_train = train.drop(train.columns[idx[0]+2],axis=1)
#new_test = test.drop(train.columns[idx[0]+1],axis=1)

In [ ]:
#new_train.to_csv('train_feature_new.csv',index=None)
#new_test.to_csv('test_feature_new.csv',index=None)

In [ ]:
#fig, ax = plt.subplots(figsize=(30, 30))
#plot_importance(model,ax=ax)

In [ ]:
pred = model.predict(test.drop(['uid'],axis=1))

In [ ]:
res = pd.DataFrame({'uid':test.uid,'label':pred})

In [ ]:
res = res.sort_values(by='label',ascending=False)
res.label = res.label.map(lambda x: 1 if x>=0.5 else 0)
res.label = res.label.map(lambda x: int(x))

In [ ]:
res.to_csv('res.csv',index=False,header=False,sep=',',columns=['uid','label'])